In [41]:
import time
import numpy as np
import pandas as pd
from dateutil.parser import parse
from datetime import date, timedelta
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn import *

In [2]:
data_path = 'data/'

air_reserve = pd.read_csv(data_path + 'air_reserve.csv').rename(columns={'air_store_id':'store_id'})
hpg_reserve = pd.read_csv(data_path + 'hpg_reserve.csv').rename(columns={'hpg_store_id':'store_id'})
air_store = pd.read_csv(data_path + 'air_store_info.csv').rename(columns={'air_store_id':'store_id'})
hpg_store = pd.read_csv(data_path + 'hpg_store_info.csv').rename(columns={'hpg_store_id':'store_id'})
air_visit = pd.read_csv(data_path + 'air_visit_data.csv').rename(columns={'air_store_id':'store_id'})
store_id_map = pd.read_csv(data_path + 'store_id_relation.csv').set_index('hpg_store_id',drop=False)
date_info = pd.read_csv(data_path + 'date_info.csv').rename(columns={'calendar_date': 'visit_date'}).drop('day_of_week',axis=1)
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [3]:
submission['visit_date'] = submission['id'].str[-10:]
submission['store_id'] = submission['id'].str[:-11]
air_reserve['visit_date'] = air_reserve['visit_datetime'].str[:10]
air_reserve['reserve_date'] = air_reserve['reserve_datetime'].str[:10]
air_reserve['dow'] = pd.to_datetime(air_reserve['visit_date']).dt.dayofweek
hpg_reserve['visit_date'] = hpg_reserve['visit_datetime'].str[:10]
hpg_reserve['reserve_date'] = hpg_reserve['reserve_datetime'].str[:10]
hpg_reserve['dow'] = pd.to_datetime(hpg_reserve['visit_date']).dt.dayofweek

In [4]:
air_visit['id'] = air_visit['store_id'] + '_' + air_visit['visit_date']
air_visit

,store_id,visit_date,visitors,id
0,air_ba937bf13d40fb24,2016-01-13,25,air_ba937bf13d40fb24_2016-01-13
1,air_ba937bf13d40fb24,2016-01-14,32,air_ba937bf13d40fb24_2016-01-14
2,air_ba937bf13d40fb24,2016-01-15,29,air_ba937bf13d40fb24_2016-01-15
3,air_ba937bf13d40fb24,2016-01-16,22,air_ba937bf13d40fb24_2016-01-16
4,air_ba937bf13d40fb24,2016-01-18,6,air_ba937bf13d40fb24_2016-01-18
...,...,...,...,...
252103,air_24e8414b9b07decb,2017-04-18,6,air_24e8414b9b07decb_2017-04-18
252104,air_24e8414b9b07decb,2017-04-19,6,air_24e8414b9b07decb_2017-04-19
252105,air_24e8414b9b07decb,2017-04-20,7,air_24e8414b9b07decb_2017-04-20
252106,air_24e8414b9b07decb,2017-04-21,8,air_24e8414b9b07decb_2017-04-21


In [5]:
submission

,id,visitors,visit_date,store_id
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9
...,...,...,...,...
32014,air_fff68b929994bfbd_2017-05-27,0,2017-05-27,air_fff68b929994bfbd
32015,air_fff68b929994bfbd_2017-05-28,0,2017-05-28,air_fff68b929994bfbd
32016,air_fff68b929994bfbd_2017-05-29,0,2017-05-29,air_fff68b929994bfbd
32017,air_fff68b929994bfbd_2017-05-30,0,2017-05-30,air_fff68b929994bfbd


In [6]:
hpg_reserve

,store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date,reserve_date,dow
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1,2016-01-01,2016-01-01,4
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3,2016-01-01,2016-01-01,4
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2,2016-01-01,2016-01-01,4
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5,2016-01-01,2016-01-01,4
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13,2016-01-01,2016-01-01,4
...,...,...,...,...,...,...,...
2000315,hpg_2b293477ea4642fb,2017-05-31 20:00:00,2017-04-19 07:00:00,2,2017-05-31,2017-04-19,2
2000316,hpg_f922aeca48ca5a29,2017-05-31 20:00:00,2017-03-19 00:00:00,3,2017-05-31,2017-03-19,2
2000317,hpg_e9151de687b93aa3,2017-05-31 21:00:00,2017-04-14 20:00:00,49,2017-05-31,2017-04-14,2
2000318,hpg_fae6c96189b4a122,2017-05-31 21:00:00,2017-04-19 17:00:00,9,2017-05-31,2017-04-19,2


In [7]:
hpg_reserve['store_id'] = hpg_reserve['store_id'].map(store_id_map['air_store_id']).fillna(hpg_reserve['store_id'])

In [8]:
hpg_reserve

,store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date,reserve_date,dow
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1,2016-01-01,2016-01-01,4
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3,2016-01-01,2016-01-01,4
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2,2016-01-01,2016-01-01,4
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5,2016-01-01,2016-01-01,4
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13,2016-01-01,2016-01-01,4
...,...,...,...,...,...,...,...
2000315,hpg_2b293477ea4642fb,2017-05-31 20:00:00,2017-04-19 07:00:00,2,2017-05-31,2017-04-19,2
2000316,hpg_f922aeca48ca5a29,2017-05-31 20:00:00,2017-03-19 00:00:00,3,2017-05-31,2017-03-19,2
2000317,hpg_e9151de687b93aa3,2017-05-31 21:00:00,2017-04-14 20:00:00,49,2017-05-31,2017-04-14,2
2000318,hpg_fae6c96189b4a122,2017-05-31 21:00:00,2017-04-19 17:00:00,9,2017-05-31,2017-04-19,2


In [9]:
len(hpg_reserve['store_id'].unique())

13325

In [10]:
hpg_store

,store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
...,...,...,...,...,...
4685,hpg_c6ed353a48c7c93e,Sichuan food,Tōkyō-to Chiyoda-ku None,35.695780,139.768453
4686,hpg_a58d2423360b2fb4,Sichuan food,Tōkyō-to Minato-ku Roppongi,35.666137,139.733978
4687,hpg_d2021704076cdd70,Shanghai food,Tōkyō-to Chiyoda-ku None,35.695780,139.768453
4688,hpg_602472570d65e219,Spain/Mediterranean cuisine,Tōkyō-to Toshima-ku None,35.730068,139.711742


In [11]:
hpg_store['store_id'] = hpg_store['store_id'].map(store_id_map['air_store_id']).fillna(hpg_store['store_id'])

In [12]:
hpg_store

,store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
...,...,...,...,...,...
4685,hpg_c6ed353a48c7c93e,Sichuan food,Tōkyō-to Chiyoda-ku None,35.695780,139.768453
4686,hpg_a58d2423360b2fb4,Sichuan food,Tōkyō-to Minato-ku Roppongi,35.666137,139.733978
4687,hpg_d2021704076cdd70,Shanghai food,Tōkyō-to Chiyoda-ku None,35.695780,139.768453
4688,hpg_602472570d65e219,Spain/Mediterranean cuisine,Tōkyō-to Toshima-ku None,35.730068,139.711742


In [13]:
hpg_store.rename(columns={'hpg_genre_name':'air_genre_name','hpg_area_name':'air_area_name'},inplace=True)
hpg_store

,store_id,air_genre_name,air_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
...,...,...,...,...,...
4685,hpg_c6ed353a48c7c93e,Sichuan food,Tōkyō-to Chiyoda-ku None,35.695780,139.768453
4686,hpg_a58d2423360b2fb4,Sichuan food,Tōkyō-to Minato-ku Roppongi,35.666137,139.733978
4687,hpg_d2021704076cdd70,Shanghai food,Tōkyō-to Chiyoda-ku None,35.695780,139.768453
4688,hpg_602472570d65e219,Spain/Mediterranean cuisine,Tōkyō-to Toshima-ku None,35.730068,139.711742


In [14]:
air_store

,store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
...,...,...,...,...,...
824,air_9bf595ef095572fb,International cuisine,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051
825,air_764f71040a413d4d,Asian,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051
826,air_10bbe8acd943d8f6,Asian,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051
827,air_7514d90009613cd6,Karaoke/Party,Hokkaidō Sapporo-shi Minami 3 Jōnishi,43.055460,141.340956


In [15]:
air_store['air_area_name0'] = air_store['air_area_name'].apply(lambda x: x.split(' ')[0])
air_store

,store_id,air_genre_name,air_area_name,latitude,longitude,air_area_name0
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,Hyōgo-ken
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,Hyōgo-ken
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,Hyōgo-ken
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,Hyōgo-ken
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō-to
...,...,...,...,...,...,...
824,air_9bf595ef095572fb,International cuisine,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051,Tōkyō-to
825,air_764f71040a413d4d,Asian,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051,Tōkyō-to
826,air_10bbe8acd943d8f6,Asian,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051,Tōkyō-to
827,air_7514d90009613cd6,Karaoke/Party,Hokkaidō Sapporo-shi Minami 3 Jōnishi,43.055460,141.340956,Hokkaidō


In [16]:
lbl = LabelEncoder()
air_store

,store_id,air_genre_name,air_area_name,latitude,longitude,air_area_name0
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,Hyōgo-ken
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,Hyōgo-ken
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,Hyōgo-ken
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,Hyōgo-ken
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō-to
...,...,...,...,...,...,...
824,air_9bf595ef095572fb,International cuisine,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051,Tōkyō-to
825,air_764f71040a413d4d,Asian,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051,Tōkyō-to
826,air_10bbe8acd943d8f6,Asian,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051,Tōkyō-to
827,air_7514d90009613cd6,Karaoke/Party,Hokkaidō Sapporo-shi Minami 3 Jōnishi,43.055460,141.340956,Hokkaidō


In [17]:
air_store['air_genre_name'] = lbl.fit_transform(air_store['air_genre_name'])
air_store

,store_id,air_genre_name,air_area_name,latitude,longitude,air_area_name0
0,air_0f0cdeee6c9bf3d7,6,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,Hyōgo-ken
1,air_7cc17a324ae5c7dc,6,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,Hyōgo-ken
2,air_fee8dcf4d619598e,6,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,Hyōgo-ken
3,air_a17f0778617c76e2,6,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,Hyōgo-ken
4,air_83db5aff8f50478e,6,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō-to
...,...,...,...,...,...,...
824,air_9bf595ef095572fb,5,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051,Tōkyō-to
825,air_764f71040a413d4d,0,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051,Tōkyō-to
826,air_10bbe8acd943d8f6,0,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051,Tōkyō-to
827,air_7514d90009613cd6,9,Hokkaidō Sapporo-shi Minami 3 Jōnishi,43.055460,141.340956,Hokkaidō


In [18]:
air_store['air_area_name0'] = lbl.fit_transform(air_store['air_area_name0'])
air_store

,store_id,air_genre_name,air_area_name,latitude,longitude,air_area_name0
0,air_0f0cdeee6c9bf3d7,6,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,3
1,air_7cc17a324ae5c7dc,6,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,3
2,air_fee8dcf4d619598e,6,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,3
3,air_a17f0778617c76e2,6,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,3
4,air_83db5aff8f50478e,6,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7
...,...,...,...,...,...,...
824,air_9bf595ef095572fb,5,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051,7
825,air_764f71040a413d4d,0,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051,7
826,air_10bbe8acd943d8f6,0,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051,7
827,air_7514d90009613cd6,9,Hokkaidō Sapporo-shi Minami 3 Jōnishi,43.055460,141.340956,2


In [19]:
date_info

,visit_date,holiday_flg
0,2016-01-01,1
1,2016-01-02,1
2,2016-01-03,1
3,2016-01-04,0
4,2016-01-05,0
...,...,...
512,2017-05-27,0
513,2017-05-28,0
514,2017-05-29,0
515,2017-05-30,0


In [20]:
date_info['holiday_flg2'] = pd.to_datetime(date_info['visit_date']).dt.dayofweek
date_info['holiday_flg2'] = ((date_info['holiday_flg2']>4) | (date_info['holiday_flg']==1)).astype(int)
date_info

,visit_date,holiday_flg,holiday_flg2
0,2016-01-01,1,1
1,2016-01-02,1,1
2,2016-01-03,1,1
3,2016-01-04,0,0
4,2016-01-05,0,0
...,...,...,...
512,2017-05-27,0,1
513,2017-05-28,0,1
514,2017-05-29,0,0
515,2017-05-30,0,0


In [21]:
data = pd.concat([air_visit, submission]).copy()
data['dow'] = pd.to_datetime(data['visit_date']).dt.dayofweek
data

,store_id,visit_date,visitors,id,dow
0,air_ba937bf13d40fb24,2016-01-13,25,air_ba937bf13d40fb24_2016-01-13,2
1,air_ba937bf13d40fb24,2016-01-14,32,air_ba937bf13d40fb24_2016-01-14,3
2,air_ba937bf13d40fb24,2016-01-15,29,air_ba937bf13d40fb24_2016-01-15,4
3,air_ba937bf13d40fb24,2016-01-16,22,air_ba937bf13d40fb24_2016-01-16,5
4,air_ba937bf13d40fb24,2016-01-18,6,air_ba937bf13d40fb24_2016-01-18,0
...,...,...,...,...,...
32014,air_fff68b929994bfbd,2017-05-27,0,air_fff68b929994bfbd_2017-05-27,5
32015,air_fff68b929994bfbd,2017-05-28,0,air_fff68b929994bfbd_2017-05-28,6
32016,air_fff68b929994bfbd,2017-05-29,0,air_fff68b929994bfbd_2017-05-29,0
32017,air_fff68b929994bfbd,2017-05-30,0,air_fff68b929994bfbd_2017-05-30,1


In [22]:
data['visitors'] = np.log1p(data['visitors'])
data

,store_id,visit_date,visitors,id,dow
0,air_ba937bf13d40fb24,2016-01-13,3.258097,air_ba937bf13d40fb24_2016-01-13,2
1,air_ba937bf13d40fb24,2016-01-14,3.496508,air_ba937bf13d40fb24_2016-01-14,3
2,air_ba937bf13d40fb24,2016-01-15,3.401197,air_ba937bf13d40fb24_2016-01-15,4
3,air_ba937bf13d40fb24,2016-01-16,3.135494,air_ba937bf13d40fb24_2016-01-16,5
4,air_ba937bf13d40fb24,2016-01-18,1.945910,air_ba937bf13d40fb24_2016-01-18,0
...,...,...,...,...,...
32014,air_fff68b929994bfbd,2017-05-27,0.000000,air_fff68b929994bfbd_2017-05-27,5
32015,air_fff68b929994bfbd,2017-05-28,0.000000,air_fff68b929994bfbd_2017-05-28,6
32016,air_fff68b929994bfbd,2017-05-29,0.000000,air_fff68b929994bfbd_2017-05-29,0
32017,air_fff68b929994bfbd,2017-05-30,0.000000,air_fff68b929994bfbd_2017-05-30,1


In [23]:
data2 = data.merge(air_store,on='store_id',how='left')
data2

,store_id,visit_date,visitors,id,dow,air_genre_name,air_area_name,latitude,longitude,air_area_name0
0,air_ba937bf13d40fb24,2016-01-13,3.258097,air_ba937bf13d40fb24_2016-01-13,2,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7
1,air_ba937bf13d40fb24,2016-01-14,3.496508,air_ba937bf13d40fb24_2016-01-14,3,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7
2,air_ba937bf13d40fb24,2016-01-15,3.401197,air_ba937bf13d40fb24_2016-01-15,4,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7
3,air_ba937bf13d40fb24,2016-01-16,3.135494,air_ba937bf13d40fb24_2016-01-16,5,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7
4,air_ba937bf13d40fb24,2016-01-18,1.945910,air_ba937bf13d40fb24_2016-01-18,0,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7
...,...,...,...,...,...,...,...,...,...,...
284122,air_fff68b929994bfbd,2017-05-27,0.000000,air_fff68b929994bfbd_2017-05-27,5,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7
284123,air_fff68b929994bfbd,2017-05-28,0.000000,air_fff68b929994bfbd_2017-05-28,6,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7
284124,air_fff68b929994bfbd,2017-05-29,0.000000,air_fff68b929994bfbd_2017-05-29,0,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7
284125,air_fff68b929994bfbd,2017-05-30,0.000000,air_fff68b929994bfbd_2017-05-30,1,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7


In [24]:
data2 = data2.merge(date_info[['visit_date','holiday_flg','holiday_flg2']], on=['visit_date'],how='left')
data2

,store_id,visit_date,visitors,id,dow,air_genre_name,air_area_name,latitude,longitude,air_area_name0,holiday_flg,holiday_flg2
0,air_ba937bf13d40fb24,2016-01-13,3.258097,air_ba937bf13d40fb24_2016-01-13,2,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0
1,air_ba937bf13d40fb24,2016-01-14,3.496508,air_ba937bf13d40fb24_2016-01-14,3,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0
2,air_ba937bf13d40fb24,2016-01-15,3.401197,air_ba937bf13d40fb24_2016-01-15,4,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0
3,air_ba937bf13d40fb24,2016-01-16,3.135494,air_ba937bf13d40fb24_2016-01-16,5,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,1
4,air_ba937bf13d40fb24,2016-01-18,1.945910,air_ba937bf13d40fb24_2016-01-18,0,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
284122,air_fff68b929994bfbd,2017-05-27,0.000000,air_fff68b929994bfbd_2017-05-27,5,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,1
284123,air_fff68b929994bfbd,2017-05-28,0.000000,air_fff68b929994bfbd_2017-05-28,6,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,1
284124,air_fff68b929994bfbd,2017-05-29,0.000000,air_fff68b929994bfbd_2017-05-29,0,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,0
284125,air_fff68b929994bfbd,2017-05-30,0.000000,air_fff68b929994bfbd_2017-05-30,1,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,0


In [25]:
data2.isna().sum()

store_id          0
visit_date        0
visitors          0
id                0
dow               0
air_genre_name    0
air_area_name     0
latitude          0
longitude         0
air_area_name0    0
holiday_flg       0
holiday_flg2      0
dtype: int64

In [29]:
air_reserve_temp = air_reserve.copy()

In [30]:
air_reserve_temp = air_reserve_temp.merge(air_store,on='store_id',how='left')

In [31]:
air_reserve_temp.isnull().sum()

store_id            0
visit_datetime      0
reserve_datetime    0
reserve_visitors    0
visit_date          0
reserve_date        0
dow                 0
air_genre_name      0
air_area_name       0
latitude            0
longitude           0
air_area_name0      0
dtype: int64

In [32]:
air_reserve_temp['diff_time'] = (pd.to_datetime(air_reserve['visit_datetime'])-pd.to_datetime(air_reserve['reserve_datetime'])).dt.days

In [33]:
air_reserve_temp

,store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date,reserve_date,dow,air_genre_name,air_area_name,latitude,longitude,air_area_name0,diff_time
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016-01-01,2016-01-01,4,8,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,2016-01-01,2016-01-01,4,4,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,8,0
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,2016-01-01,2016-01-01,4,4,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,8,0
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,2016-01-01,2016-01-01,4,8,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,2016-01-01,2016-01-01,4,4,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92373,air_db4b38ebe7a7ceff,2017-05-29 18:00:00,2017-04-17 19:00:00,6,2017-05-29,2017-04-17,0,4,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,8,41
92374,air_1033310359ceeac1,2017-05-30 23:00:00,2017-04-19 17:00:00,8,2017-05-30,2017-04-19,1,7,Tōkyō-to Kōtō-ku Tomioka,35.671270,139.796987,7,41
92375,air_877f79706adbfb06,2017-05-31 20:00:00,2017-04-12 23:00:00,3,2017-05-31,2017-04-12,2,8,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,48
92376,air_900d755ebd2f7bbd,2017-05-31 20:00:00,2017-04-19 16:00:00,10,2017-05-31,2017-04-19,2,6,Tōkyō-to Chūō-ku Ginza,35.672114,139.770825,7,42


In [34]:
data2['year'] = data2['visit_date'].str[:4].astype(int)
data2['month'] = data2['visit_date'].str[5:7].astype(int)
data2['day'] = data2['visit_date'].str[8:].astype(int)
data2

,store_id,visit_date,visitors,id,dow,air_genre_name,air_area_name,latitude,longitude,air_area_name0,holiday_flg,holiday_flg2,year,month,day
0,air_ba937bf13d40fb24,2016-01-13,3.258097,air_ba937bf13d40fb24_2016-01-13,2,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0,2016,1,13
1,air_ba937bf13d40fb24,2016-01-14,3.496508,air_ba937bf13d40fb24_2016-01-14,3,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0,2016,1,14
2,air_ba937bf13d40fb24,2016-01-15,3.401197,air_ba937bf13d40fb24_2016-01-15,4,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0,2016,1,15
3,air_ba937bf13d40fb24,2016-01-16,3.135494,air_ba937bf13d40fb24_2016-01-16,5,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,1,2016,1,16
4,air_ba937bf13d40fb24,2016-01-18,1.945910,air_ba937bf13d40fb24_2016-01-18,0,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0,2016,1,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284122,air_fff68b929994bfbd,2017-05-27,0.000000,air_fff68b929994bfbd_2017-05-27,5,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,1,2017,5,27
284123,air_fff68b929994bfbd,2017-05-28,0.000000,air_fff68b929994bfbd_2017-05-28,6,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,1,2017,5,28
284124,air_fff68b929994bfbd,2017-05-29,0.000000,air_fff68b929994bfbd_2017-05-29,0,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,0,2017,5,29
284125,air_fff68b929994bfbd,2017-05-30,0.000000,air_fff68b929994bfbd_2017-05-30,1,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,0,2017,5,30


In [35]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284127 entries, 0 to 284126
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   store_id        284127 non-null  object 
 1   visit_date      284127 non-null  object 
 2   visitors        284127 non-null  float64
 3   id              284127 non-null  object 
 4   dow             284127 non-null  int64  
 5   air_genre_name  284127 non-null  int32  
 6   air_area_name   284127 non-null  object 
 7   latitude        284127 non-null  float64
 8   longitude       284127 non-null  float64
 9   air_area_name0  284127 non-null  int32  
 10  holiday_flg     284127 non-null  int64  
 11  holiday_flg2    284127 non-null  int32  
 12  year            284127 non-null  int32  
 13  month           284127 non-null  int32  
 14  day             284127 non-null  int32  
dtypes: float64(3), int32(6), int64(2), object(4)
memory usage: 28.2+ MB


In [36]:
data2['store_id_int'] = lbl.fit_transform(data2['store_id'])
data2

,store_id,visit_date,visitors,id,dow,air_genre_name,air_area_name,latitude,longitude,air_area_name0,holiday_flg,holiday_flg2,year,month,day,store_id_int
0,air_ba937bf13d40fb24,2016-01-13,3.258097,air_ba937bf13d40fb24_2016-01-13,2,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0,2016,1,13,603
1,air_ba937bf13d40fb24,2016-01-14,3.496508,air_ba937bf13d40fb24_2016-01-14,3,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0,2016,1,14,603
2,air_ba937bf13d40fb24,2016-01-15,3.401197,air_ba937bf13d40fb24_2016-01-15,4,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0,2016,1,15,603
3,air_ba937bf13d40fb24,2016-01-16,3.135494,air_ba937bf13d40fb24_2016-01-16,5,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,1,2016,1,16,603
4,air_ba937bf13d40fb24,2016-01-18,1.945910,air_ba937bf13d40fb24_2016-01-18,0,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0,2016,1,18,603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284122,air_fff68b929994bfbd,2017-05-27,0.000000,air_fff68b929994bfbd_2017-05-27,5,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,1,2017,5,27,828
284123,air_fff68b929994bfbd,2017-05-28,0.000000,air_fff68b929994bfbd_2017-05-28,6,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,1,2017,5,28,828
284124,air_fff68b929994bfbd,2017-05-29,0.000000,air_fff68b929994bfbd_2017-05-29,0,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,0,2017,5,29,828
284125,air_fff68b929994bfbd,2017-05-30,0.000000,air_fff68b929994bfbd_2017-05-30,1,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,0,2017,5,30,828


In [39]:
train_size = len(air_visit)
train = data2[:train_size]
test = data2[train_size:]

In [42]:
col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors']]
train = train.fillna(-1)
test = test.fillna(-1)

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5
    
model1 = ensemble.GradientBoostingRegressor(learning_rate=0.2, random_state=3, n_estimators=200, subsample=0.8, 
                      max_depth =10)
model2 = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)
model3 = XGBRegressor(learning_rate=0.2, random_state=3, n_estimators=280, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =12)